In [ ]:
# Jared Gibbs
# Truth in Taxation Inflation Rate Adjustment Modeling App
# 8/2/2025

In [ ]:
### Resources

# Hvplot referance gallery: https://hvplot.holoviz.org/reference/index.html
    # look here for creating interative graph/ chart objects

# Panel component gallery: https://panel.holoviz.org/reference/index.html#templates
    # look here for widget and layout component guides

In [ ]:
### Questions

# How do we deal with personal property rates since they are a 1-year lag of real property rates?
# How do we deal with new entities/ rates being created?

# inflate rate vs inflate revenue? 

# how to model the 10% growth cap? 

In [ ]:
### Libraries
# Be sure to download these libraries if you don't already have them
%pip install datashader; hvplot; param; holoviews; geoviews; colorcet; openpyxl; xlrd

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import datashader, hvplot, param, holoviews as hv
import geoviews as gv
import panel as pn
import colorcet as cc
from colorcet.plotting import swatch, swatches
hv.extension('bokeh', 'matplotlib')
import hvplot.pandas
import pathlib


In [ ]:
### Data Set Up

# Read in the data
df_raw_inflation = pd.read_excel('Trimmed Mean PCE Inflation.xlsx')
df_raw_property = pd.read_excel('document (79).xls')

df_cleandata = df_raw_property[["Tax Year", "County Name", "Entity Name", "750 Tax_Rate_Real", "750 Tax_Rate_PP", "233b Total_Real", "233b Total_PP", "233b Total_CA"]]

# Initialize Global Lists and Variables

entities = df_cleandata['Entity Name'].unique().tolist()
years = df_cleandata["Tax Year"].unique().tolist()

adjustment_factor= 1000000
rate_growth_cap = 10

# Calculate Base Revenue
df_cleandata['base_total_property'] = df_cleandata["233b Total_Real"] + df_cleandata["233b Total_CA"] # + df_cleandata["233b Total_PP"] 

df_cleandata['base_real_revenue'] = (df_cleandata['750 Tax_Rate_Real']/ adjustment_factor)* df_cleandata['233b Total_Real']
#df_cleandata['base_pp_revenue'] = df_cleandata['750 Tax_Rate_PP']* df_cleandata['233b Total_PP']
df_cleandata['base_ca_revenue'] = (df_cleandata['750 Tax_Rate_Real']/ adjustment_factor)* df_cleandata['233b Total_CA']
df_cleandata['base_total_revenue'] = df_cleandata['base_real_revenue'] + df_cleandata['base_ca_revenue'] # + df_cleandata['base_pp_revenue'] 


### Compute Statewide variables

variables = df_cleandata.columns.to_list()
variables = variables[5:]

df_cleandata['entity_code']=df_cleandata['Entity Name'].str[0:4]
df_cleandata['entity_code']=df_cleandata['entity_code'].astype(int)

df_cleandata['Entity Name'] = df_cleandata['Entity Name'].str[5:]


for year in years:
    df_cleandata.loc[-1]= [year, "99-STATEWIDE", "9999-STATEWIDE", 0, 0, 
                        0, 0, 0, 0, 0, 0, 0, 9999]
    
    for variable in variables:

        alpha = df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['entity_code'] == 1010 )][variable].sum()

        df_cleandata.loc[[-1],[variable]] = alpha

    df_cleandata.index = df_cleandata.index + 1



df_cleandata = df_cleandata.merge(df_raw_inflation, how='left', on="Tax Year")


# inflation_rate_forwardlag for trimmed mean annual PCE inflation
# inflation_rate_rolling for 5-year rolling average of CPI for urban less food and energy

df_cleandata['inflation_rate_forwardlag'] = (df_cleandata['inflation_rate_forwardlag']/100) + 1

entities = df_cleandata['Entity Name'].unique().tolist()

#Set up colors and colormaping
highlight = '#196B24'
alt_highlight ='#8b4aa8'

base_colors = ['#196B24', '#8b4aa8', '#767167', '#bcb4a5', '#171615',  '#2f2d29', '#47443e', '#5e5a52', '#8d877c', '#a59e90', '#d4cbb9', '#ebe1ce']
swatch(name= 'base_colors', cmap = base_colors)

In [ ]:
# Inflate Revenue 
# real property 
for year in years:
    for entity in entities:
        if year == 2015:
            gamma = df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["base_real_revenue"].sum() * df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["inflation_rate_forwardlag"].sum()
            df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity),"direct-inflated_real_revenue"] = gamma
        
        if year >= 2016:
            gamma = df_cleandata.loc[(df_cleandata["Tax Year"] == year - 1) & (df_cleandata['Entity Name']== entity)]["direct-inflated_real_revenue"].sum() * df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["inflation_rate_forwardlag"].sum()
            df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity),"direct-inflated_real_revenue"] = gamma

# personal property 
'''
for year in years:
    for entity in entities:
        if year == 2015:
            gamma = df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["base_pp_revenue"].sum() * df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["inflation_rate_forwardlag"].sum()
            df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity),"direct-inflated_pp_revenue"] = gamma
        
        if year >= 2016:
            gamma = df_cleandata.loc[(df_cleandata["Tax Year"] == year - 1) & (df_cleandata['Entity Name']== entity)]["direct-inflated_pp_revenue"].sum() * df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["inflation_rate_forwardlag"].sum()
            df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity),"direct-inflated_pp_revenue"] = gamma
'''

# Cenrally Assesed property 
for year in years:
    for entity in entities:
        if year == 2015:
            gamma = df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["base_ca_revenue"].sum() * df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["inflation_rate_forwardlag"].sum()
            df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity),"direct-inflated_ca_revenue"] = gamma
        
        if year >= 2016:
            gamma = df_cleandata.loc[(df_cleandata["Tax Year"] == year - 1) & (df_cleandata['Entity Name']== entity)]["direct-inflated_ca_revenue"].sum() * df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["inflation_rate_forwardlag"].sum()
            df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity),"direct-inflated_ca_revenue"] = gamma



In [ ]:
# Compute direct-inflated differential
df_cleandata['direct-inflated_diff_real'] = df_cleandata['direct-inflated_real_revenue']- df_cleandata['base_real_revenue']
#df_cleandata['direct-inflated_diff_pp'] = df_cleandata['direct-inflated_pp_revenue']- df_cleandata['base_pp_revenue']
df_cleandata['direct-inflated_diff_ca'] = df_cleandata['direct-inflated_ca_revenue']- df_cleandata['base_ca_revenue']

df_cleandata['direct-inflated_total_revenue'] = df_cleandata['direct-inflated_real_revenue'] + df_cleandata['direct-inflated_ca_revenue'] # + df_cleandata['direct-inflated_diff_pp']
df_cleandata['direct-inflated_diff_total'] = df_cleandata['direct-inflated_total_revenue']- df_cleandata['base_total_revenue']


In [ ]:
# Compute Observed Growth Rates
for year in years:
    for entity in entities:
        if year == 2014:
            df_cleandata['observed_growth_rate'] = 0
        
        if year >= 2015:
            df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity),'observed_growth_rate'] = ((df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["base_total_revenue"].sum()/df_cleandata.loc[(df_cleandata["Tax Year"] == year -1) & (df_cleandata['Entity Name']== entity)]["base_total_revenue"].sum())-1)*100

df_cleandata['inflation_rate'] = (df_cleandata['inflation_rate_forwardlag']-1)*100

In [ ]:
# Capped growth scenario

# compounded 10% growth of total revenue
df_cleandata['10-growth_total_revenue'] = df_cleandata['base_total_revenue']

for year in years:
    for entity in entities:
        
        if year >= 2015:
            delta = df_cleandata.loc[(df_cleandata["Tax Year"] == year- 1 ) & (df_cleandata['Entity Name']== entity)]["10-growth_total_revenue"].sum() * ((rate_growth_cap/100)+1)
            df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity),"10-growth_total_revenue"] = delta

# Capped growth rate
df_cleandata["cap_growth_rate"] = df_cleandata['observed_growth_rate']

for year in years:
    for entity in entities:
        if df_cleandata.loc[(df_cleandata["Tax Year"] == year ) & (df_cleandata['Entity Name']== entity)]["observed_growth_rate"].sum() > rate_growth_cap:
            df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity),"cap_growth_rate"] = rate_growth_cap

# Capped growth total revenue
df_cleandata['cap-growth_total_revenue'] = df_cleandata['base_total_revenue']

for year in years:
    for entity in entities:
        if year == 2015:
            zeta = df_cleandata.loc[(df_cleandata["Tax Year"] == year- 1 ) & (df_cleandata['Entity Name']== entity)]["base_total_revenue"].sum() * ((df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["cap_growth_rate"].sum()/100)+1)
            df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity),"cap-growth_total_revenue"] = zeta
        
        if year >= 2016:
            zeta = df_cleandata.loc[(df_cleandata["Tax Year"] == year- 1 ) & (df_cleandata['Entity Name']== entity)]["cap-growth_total_revenue"].sum() * ((df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity)]["cap_growth_rate"].sum()/100)+1)
            df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity),"cap-growth_total_revenue"] = zeta


# Capped growth tax rates
for year in years:
    for entity in entities:
        eta = df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata["Entity Name"]== entity)]['cap-growth_total_revenue'].sum()/ (df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata["Entity Name"]== entity)]['base_total_property'].sum()/ adjustment_factor)
        df_cleandata.loc[(df_cleandata["Tax Year"] == year) & (df_cleandata['Entity Name']== entity),"cap-growth_tax_rate"] = eta


In [ ]:
# Compute direct-inflated tax rates
df_cleandata['inflated_eff_tax_rate'] = df_cleandata['direct-inflated_total_revenue']/(df_cleandata['base_total_property']/ adjustment_factor)

In [ ]:
# Transform Data to a 'reactive object'. 
    # This sets up the interativity with widgets
df_working = pn.rx(df_cleandata)


In [ ]:
### Set Up Widgets

entites = df_cleandata['Entity Name'].unique().tolist()
tax_year_widget = pn.widgets.EditableIntSlider(name = 'Tax Year', fixed_start= 2014, fixed_end = 2024, step = 1, value = 2024 )

# Widgets

entitiy_widget = pn.widgets.AutocompleteInput(name = "Entitiy", options = entites, case_sensitive = False, value = 'BEAVER')



# Page Buttons. 
    #These are special widgets which are keyed to the mapping in the 'Build App Layout' chunk. If you change things here, make sure they will still match up with what appears there.
page1_button = pn.widgets.Button(name = "Revenue Comparison", 
                                     icon = 'home', 
                                     icon_size = '24px', 
                                     height = 50, 
                                     styles={"width": "100%", "border-style": "outset"})

page2_button = pn.widgets.Button(name = 'Growth Rate Comparison', 
                                         icon = 'file-info', 
                                         icon_size = '24px', 
                                         height = 50, 
                                         styles={"width": "100%", "border-style": "outset"})

page3_button = pn.widgets.Button(name = "Tax Rate Comparison",
                                          icon = 'layers-difference', 
                                          icon_size = '24px', 
                                          height = 50, 
                                          styles={"width": "100%", "border-style": "outset"})






In [ ]:
### Build Interative Objects

# Revenue timeseries
revenue_timeseries = df_working[df_working['Entity Name'] == entitiy_widget].hvplot.line(
    title = 'Property Tax Revenue',
    x = 'Tax Year',
    y = ['base_total_revenue', 'direct-inflated_total_revenue', 'cap-growth_total_revenue'],
    ylabel = "Total Revenue (M)",
    xlabel = "Tax Year",
    color = base_colors,
    line_width = 5,
    line_dash = ['solid','dashed', 'dotted'],
    fontsize={
        'title': 20,
        'xlabel': 16,
        'ylabel': 16,
        'xticks': 16,
        'yticks': 16,
        'legend': 16
    },
    responsive = True)

# Growth Rate timeseries
growth_rate_timeseries = df_working[df_working['Entity Name'] == entitiy_widget].hvplot.line(
    title = 'Growth Rates',
    x = 'Tax Year',
    y = ['observed_growth_rate', 'inflation_rate', 'cap_growth_rate'],
    ylabel = "Growth Rate (%)",
    xlabel = "Tax Year",
    color = base_colors,
    line_width = 5,
    line_dash = ['solid','dashed', 'dotted'],
    fontsize={
        'title': 20,
        'xlabel': 16,
        'ylabel': 16,
        'xticks': 16,
        'yticks': 16,
        'legend': 16
    },
    responsive = True)

# Tax Rate timeseries
tax_rate_timeseries = df_working[df_working['Entity Name'] == entitiy_widget].hvplot.line(
    title = 'Tax Rates',
    x = 'Tax Year',
    y = ['750 Tax_Rate_Real', 'inflated_eff_tax_rate', 'cap-growth_tax_rate'],
    ylabel = "Tax Rate",
    xlabel = "Tax Year",
    color = base_colors,
    line_width = 5,
    line_dash = ['solid','dashed', 'dotted'],
    fontsize={
        'title': 20,
        'xlabel': 16,
        'ylabel': 16,
        'xticks': 16,
        'yticks': 16,
        'legend': 16
    },
    responsive = True)


# Indicators
indicator_growth_rate = df_working.loc[(df_working["Entity Name"] == entitiy_widget), 'observed_growth_rate'].mean()
indicator_inflation_rate = df_working.loc[(df_working["Entity Name"] == entitiy_widget), 'inflation_rate'].mean()
indicator_cap_rate = df_working.loc[(df_working["Entity Name"] == entitiy_widget), 'cap_growth_rate'].mean()

indicator_base_revenue = df_working.loc[(df_working["Entity Name"] == entitiy_widget) & (df_working['Tax Year'] == tax_year_widget), 'base_total_revenue'].max()
indicator_inf_revenue = df_working.loc[(df_working["Entity Name"] == entitiy_widget) & (df_working['Tax Year'] == tax_year_widget), 'direct-inflated_total_revenue'].max()
indicator_cap_revenue = df_working.loc[(df_working["Entity Name"] == entitiy_widget) & (df_working['Tax Year'] == tax_year_widget), 'cap-growth_total_revenue'].max()

indicator_tax_rate = df_working.loc[(df_working["Entity Name"] == entitiy_widget) & (df_working['Tax Year'] == tax_year_widget), '750 Tax_Rate_Real'].max()
indicator_inf_rate = df_working.loc[(df_working["Entity Name"] == entitiy_widget) & (df_working['Tax Year'] == tax_year_widget), 'inflated_eff_tax_rate'].max()
indicator_cap_rate = df_working.loc[(df_working["Entity Name"] == entitiy_widget) & (df_working['Tax Year'] == tax_year_widget), 'cap-growth_tax_rate'].max()



In [ ]:
### Build Panes
indicators_revenue = pn.Column(
    pn.Card(pn.indicators.Number(value = indicator_base_revenue, format = '{value:.3f}'), title = 'Base Revenue'),
    pn.Card(pn.indicators.Number(value = indicator_inf_revenue, format = '{value:.3f}'), title = 'Inflated Revenue'),
    pn.Card(pn.indicators.Number(value = indicator_cap_revenue, format = '{value:.3f}'), title = 'Capped Revenue'),
)
indicators_growth = pn.Column(
    pn.Card(pn.indicators.Number(value = indicator_growth_rate, format = '{value:.3f}%'), title = 'Average Growth Rate'),
    pn.Card(pn.indicators.Number(value = indicator_inflation_rate, format = '{value:.3f}%'), title = 'Average Inflation Rate'),
    pn.Card(pn.indicators.Number(value = indicator_cap_rate, format = '{value:.3f}%'), title = 'Average Capped Growth'),
)

indicators_rate = pn.Column(
    pn.Card(pn.indicators.Number(value = indicator_tax_rate, format = '{value:.5f}'), title = 'Tax Rate'),
    pn.Card(pn.indicators.Number(value = indicator_inf_rate, format = '{value:.5f}'), title = 'Tax Rate, Inflated'),
    pn.Card(pn.indicators.Number(value = indicator_cap_rate, format = '{value:.5f}'), title = 'Tax Rate, Capped')
    )

In [ ]:
### Build App Layout 
# Build Panes

# Page Switching Function
def show_page(page_key):
    main_area.clear()
    main_area.append(mapping[page_key])

# Initalize Button Connections
    # These tell the app to 'watch' the button clicks and to trigger the page switching function accordingly.
page1_button.on_click(lambda event: show_page("page1"))
page2_button.on_click(lambda event: show_page("page2"))
page3_button.on_click(lambda event: show_page("page3"))

# Create pages
    # These functions determine what shows up on each page. After creating your interative objects, put those objects in the functions below in the matter you would like them to show up.
    # This part can take some faffing to get right so do expiriment with different layouts and panel object types.
def createpage_page1():
    return pn.Column(pn.Row(pn.Column(entitiy_widget, tax_year_widget, indicators_revenue),
                    pn.Card(pn.pane.HoloViews(revenue_timeseries))))

def createpage_page2():
    return pn.Column(pn.Row(pn.Column(entitiy_widget, tax_year_widget, indicators_growth),
                    pn.Card(pn.pane.HoloViews(growth_rate_timeseries))))

def createpage_page3():
    return pn.Column(pn.Row(pn.Column(entitiy_widget, tax_year_widget, indicators_rate),
                    pn.Card(pn.pane.HoloViews(tax_rate_timeseries))))


# Set up page mapping
    # This is the connective tissue that links the button connections to the appropraite page creating function.
    # If more pages are desired, be sure to build another button widget for the page, a lamda event to watch that button, a page creation fucntion to call on button press, and a new item in the dictionary below to link it all together.  
mapping = {
    "page1": createpage_page1(),
    "page2": createpage_page2(),
    "page3": createpage_page3(),
}



# Set up page chunks
    # This is where the app is actually built. It takes everything that has been created above and strings it together into a servable application.

    # 'main_area' sets the default page to land on when the app is spun up.
main_area = pn.Column(mapping["page1"], styles={"width":"100%"})

    # sidebar_default' creates a sidebar object to house all the buttons for switching between pages
sidebar_default = pn.Column(pn.pane.PNG("LFA Logo w text.png", width = 250, height = 100, align = ('center', 'center')), 
                    #pn.pane.Markdown("# Pages", align = ('center', 'center')),  
                    page1_button, 
                    page2_button, 
                    page3_button, 
                    styles={"width": "100%", "padding": "15px"})

# Initalize template
pn.extension(design = 'material')

template = pn.template.MaterialTemplate(
    title = 'Truth in Taxation: Inflation Modeling',
    header_background = 'Black',
    logo = "LFA Logo.png",
    sidebar = [sidebar_default],
    sidebar_width = 400,
    main = [main_area]
)

template.show()